# News

In [ ]:
# 한국일보 뉴스 말뭉치
# http://kristalinfo.dynu.net/TestCollections/
# hkib-20000-40075.tar.gz 
# ./data/

import os
import re # 정규표현식 라이브러리
import numpy as np
import pandas as pd

from konlpy.tag import Kkma, Hannanum
from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
dir_prefix='./data/'
target_dir='HKIB-20000'
cat_dirs=['health','economy','science','education','culture','society','industry','leisure','politics']
cat_prefixes=['건강','경제','과학','교육','문화','사회','산업','여가','정치']
files=os.listdir(dir_prefix+target_dir)
files

In [ ]:
files = os.listdir(dir_prefix + target_dir)

for file in files:
    if not file.endswith('.txt'):
        continue
    
    with open(dir_prefix + target_dir + '/' + file) as currfile:
        doc_cnt = 0
        docs = []
        curr_doc = None
        
        for curr_line in currfile:
            if curr_line.startswith('@DOCUMENT'):
                if curr_doc is not None:
                    docs.append(curr_doc)
                curr_doc = curr_line
                doc_cnt = doc_cnt + 1
                continue
            curr_doc = curr_doc + curr_line
        
        for doc in docs:
            doc_lines = doc.split('\n')
            doc_no = doc_lines[1][9:]
            
            doc_cat03 = ''
            for line in doc_lines[:10]:
                if line.startswith("#CAT'03:"):
                    doc_cat03 = line[10:]
                    break
            
            for cat_prefix in cat_prefixes:
                if doc_cat03.startswith(cat_prefix):
                    dir_index = cat_prefixes.index(cat_prefix)
                    break
                    
            filtered_lines = []
            for line in doc_lines:
                if not (line.startswith('#') or line.startswith('@')):
                    filtered_lines.append(line)
                    
            filename = 'hkib-' + doc_no + '.txt'
            filepath = dir_prefix + target_dir + '/' + cat_dirs[dir_index]
            
            if not os.path.exists(filepath):
                os.makedirs(filepath)
            f = open(filepath + '/' + filename, 'w')
            f.write('\n'.join(filtered_lines))
            f.close()

In [ ]:
dirs=['education','health']
for i, d in enumerate(dirs):
    files=os.listdir(dir_prefix+target_dir+'/'+d)
    for file in files:
        f=open(dir_prefix+target_dir+'/'+d+'/'+file,'r',encoding='utf-8')
        raw=f.read()
        reg_raw = re.sub(r'[-\'@#:/◆▲0-9a-zA-Z<>!-"*\(\)]', '', raw)
        reg_raw = re.sub(r'[ ]+', ' ', reg_raw)
        reg_raw = reg_raw.replace('\n', ' ')
        
        f.close()

In [ ]:
x_ls=[]
y_ls=[]
tmp1=[]
tmp2=''
tokenizer=Kkma()

for i,d in enumerate(dirs):
    files=os.listdir(dir_prefix+target_dir+'/'+d)
    
    for file in files:
        f=open(dir_prefix+target_dir+'/'+d+'/'+file,'r',encoding='utf-8')
        raw=f.read()
        reg_raw = re.sub(r'[-\'@#:/◆▲0-9a-zA-Z<>!-"*\(\)]', '', raw)
        reg_raw = re.sub(r'[ ]+', ' ', reg_raw)
        reg_raw = reg_raw.replace('\n', ' ')
        tokens=tokenizer.nouns(reg_raw)
        print(file)
        for token in tokens:
            tmp1.append(token)
            
        tmp2=' '.join(tmp1)
        x_ls.append(tmp2)
        tmp1=[]
        
        y_ls.append(i)
        f.close()

In [ ]:
pd.DataFrame(x_ls)
x_arr=np.array(x_ls)
y_arr=np.array(y_ls)
cntvec=CountVectorizer()
x_cntvecs=cntvec.fit_transform(x_arr)
x_cntarray=x_cntvecs.toarray()
pd.DataFrame(x_cntarray)

In [ ]:
for k,v in sorted(cntvec.vocabulary_.items(),key=lambda x:x[1]):
    print(k,v)

In [ ]:
tfidf_vec=TfidfVectorizer(use_idf=True)
x_tf_vec=tfidf_vec.fit_transform(x_arr)
x_tf_arr=x_tf_vec.toarray()
pd.DataFrame(x_tf_arr)

### train

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
tx,vx,ty,vy=model_selection.train_test_split(x_tf_arr,y_arr,test_size=0.2)
print(len(tx),len(vx))
train=TensorDataset(tx,ty)
print(train[0])

train_loader=DataLoader(train, batch_size=100,shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(33572,256)
        self.fc1=nn.Linear(256,256)
        self.fc1=nn.Linear(256,256)
        self.fc1=nn.Linear(256,128)
        self.fc1=nn.Linear(128,128)
        self.fc1=nn.Linear(128,2)
        
    def forward(self,x):    
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=F.relu(self.fc4(x))
        x=F.relu(self.fc5(x))
        x=self.fc6(x)
        return F.log_softmax(x)
    
model=Net()

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.005)
for epoch in range(20):
    total_loss=0
    for tx,ty in train_loader:
        tx,ty=Variable(tx),Variable(ty)
        optimizer.zero_grad()
        output=model(tx)
        loss=criterion(output,ty)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    if (epoch+1)%10 == 0:
        print(epoch+1,total_loss)

In [ ]:
vx,vy=Variable(vx),Variable(vy)
res=torch.max(model(vx).data, 1)[1]
acc=sum(vy.numpy()==res.numpy()/len(vy.numpy()))
acc